<a href="https://colab.research.google.com/github/reccos7/ObjectDetection_YOLO/blob/main/CarelessDriving_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !git clone https://github.com/ultralytics/yolov5.git

In [ ]:
# !cd yolov5
# !pip install -r /content/yolov5/requirements.txt

In [ ]:
!pip install FER

     |████████████████████████████████| 810 kB 5.2 MB/s 
     |████████████████████████████████| 2.3 MB 54.6 MB/s 


In [ ]:
# import 설정

from glob import glob
import json
from tqdm import tqdm
from PIL import Image
import cv2
import os
from tqdm import tqdm

from google.colab.patches import cv2_imshow


In [ ]:
from fer import FER
import matplotlib.pyplot as plt 
%matplotlib inline

In [ ]:
# 다렉토리 설정

data_path = '/content/drive/MyDrive/careless_driving'
train_path = data_path + '/train'
test_path = data_path + '/test'

In [ ]:
# 다렉토리 만들기

image_directory = os.path.join(data_path, "annotated_images")
if not os.path.exists(image_directory):
     os.makedirs(image_directory)
label_text_directory = os.path.join(data_path, "labeled_texts")
if not os.path.exists(label_text_directory):
     os.makedirs(label_text_directory)
emotion_directory = os.path.join(data_path, "emotions")
if not os.path.exists(emotion_directory):
     os.makedirs(emotion_directory)
boundingbox_directory = os.path.join(data_path, "bounding_box")
if not os.path.exists(boundingbox_directory):
     os.makedirs(boundingbox_directory)

In [ ]:
boundingbox_directory1 = os.path.join(data_path, "bounding_box1")
if not os.path.exists(boundingbox_directory1):
     os.makedirs(boundingbox_directory1)
boundingbox_directory2 = os.path.join(data_path, "bounding_box2")
if not os.path.exists(boundingbox_directory2):
     os.makedirs(boundingbox_directory2)

In [ ]:
yolo_boundingbox_directory1 = os.path.join(data_path, "yolo_bounding_box1")
if not os.path.exists(yolo_boundingbox_directory1):
     os.makedirs(yolo_boundingbox_directory1)

yolo_boundingbox_directory2 = os.path.join(data_path, "yolo_bounding_box2")
if not os.path.exists(yolo_boundingbox_directory2):
     os.makedirs(yolo_boundingbox_directory2)

In [ ]:
# 라벨 및 이미지(bounding box) 저장

names = ['eye_opened', 'eye_closed', 'mouth_opened', 'mouth_closed', 'face', 'phone', 'cigar']
with open(train_path+'/labels.json') as f:
    json_data = json.load(f)
    json_anno = json_data["annotations"]
    for json_img in tqdm(json_anno):
        img_id = str(json_img['file_name'])
        txt_name = str(img_id.split('.')[0])+'.txt'
        txt_dir = os.path.join(label_text_directory, txt_name)
        img_dir = train_path + '/images/' + img_id
        f_txt = open(txt_dir, 'w')
        img = cv2.imread(img_dir, cv2.IMREAD_COLOR)
        classes = []
        for img_obj in json_img['objects']:
            class_id = str(names.index(img_obj['class']))
            img_pos = img_obj['position']
            x,y,w,h = img_pos
            img_label = img_obj['class']
            classes.append(img_label)
            cv2.rectangle(img,(x,y), (w,h), (0,255,0), 3)
        for i in range(len(classes)):
            f_txt.write('%s\n' % classes[i])
        cv2.imwrite(os.path.join(image_directory,img_id), img)

In [ ]:
# 감정 인식 저장
with open(train_path+'/labels.json') as f:
    json_data = json.load(f)
    json_anno = json_data["annotations"]
    for json_img in tqdm(json_anno):
        img_id = str(json_img['file_name'])
        img_dir = train_path + '/images/' + img_id
        txt_name = str(img_id.split('.')[0])+'.txt'
        txt_dir = os.path.join(emotion_directory, txt_name)
        f_txt = open(txt_dir, 'w')
        img = cv2.imread(img_dir, cv2.IMREAD_COLOR)
        emo_detector = FER(mtcnn=True)
        captured_emotions = emo_detector.detect_emotions(img)
        captured_emotions = 'List of emotions: '+ str(captured_emotions) +'\n'
        f_txt.write(captured_emotions)
        dominant_emotion, emotion_score = emo_detector.top_emotion(img)
        dominant_emotion = '최종 감정: '+ str(dominant_emotion)
        f_txt.write(dominant_emotion)
        emotion_score = ', 감정 점수: '+ str(emotion_score)
        f_txt.write(emotion_score)

In [ ]:
# Neutral 감정 이미지 수

with open(train_path+'/labels.json') as f:
    json_data = json.load(f)
    json_anno = json_data["annotations"]
    count = 0
    for json_img in tqdm(json_anno):
        img_id = str(json_img['file_name'])
        img_dir = train_path + '/images/' + img_id
        img = cv2.imread(img_dir, cv2.IMREAD_COLOR)
        emo_detector = FER(mtcnn=True)
        dominant_emotion, emotion_score = emo_detector.top_emotion(img)
        dominant_emotion = str(dominant_emotion)
        if dominant_emotion == 'neutral':
            count+=1
            print('Found {} neutral images!'.format(count))
    print(count)

In [ ]:
# Yolo Style로 바꾸기
def convert(size, box): 
    dw = 1./size[0] 
    dh = 1./size[1] 
    x = (box[0] + box[1])/2.0 
    y = (box[2] + box[3])/2.0 
    w = box[1] - box[0] 
    h = box[3] - box[2] 
    x = x*dw 
    w = w*dw 
    y = y*dh 
    h = h*dh 
    return (x,y,w,h)


In [ ]:
# Yolo bounding box1 저장

names = ['eye_opened', 'eye_closed', 'mouth_opened', 'mouth_closed', 'face', 'phone', 'cigar']
with open(train_path+'/labels.json') as f:
    json_data = json.load(f)
    json_anno = json_data["annotations"]
    json_anno = json_anno[:8000]
    for json_img in tqdm(json_anno):
        img_id = str(json_img['file_name'])
        txt_name = str(img_id.split('.')[0])+'.txt'
        txt_dir = os.path.join(yolo_boundingbox_directory1, txt_name)
        img_dir = train_path + '/images/' + img_id
        f_txt = open(txt_dir, 'w')
        img = cv2.imread(img_dir, cv2.IMREAD_COLOR)
        w = img.shape[1]
        h = img.shape[0]
        classes = []
        for img_obj in json_img['objects']:
            class_id = str(names.index(img_obj['class'])) + " "
            f_txt.write(class_id)
            img_pos = img_obj['position']
            a,b,c,d = img_pos
            img_pos = convert((w,h),(a,c,b,d))
            for i in range(len(img_pos)):
                input = str(img_pos[i])
                f_txt.write(input)
                f_txt.write(' ')
            f_txt.write('\n')


100%|██████████| 8000/8000 [35:27<00:00,  3.76it/s]


In [ ]:
# yolo bounding box2 저장

names = ['eye_opened', 'eye_closed', 'mouth_opened', 'mouth_closed', 'face', 'phone', 'cigar']
with open(train_path+'/labels.json') as f:
    json_data = json.load(f)
    json_anno = json_data["annotations"]
    json_anno = json_anno[8000:]
    for json_img in tqdm(json_anno):
        img_id = str(json_img['file_name'])
        txt_name = str(img_id.split('.')[0])+'.txt'
        txt_dir = os.path.join(yolo_boundingbox_directory2, txt_name)
        img_dir = train_path + '/images/' + img_id
        f_txt = open(txt_dir, 'w')
        img = cv2.imread(img_dir, cv2.IMREAD_COLOR)
        w = img.shape[1]
        h = img.shape[0]
        classes = []
        for img_obj in json_img['objects']:
            class_id = str(names.index(img_obj['class'])) + " "
            f_txt.write(class_id)
            img_pos = img_obj['position']
            a,b,c,d = img_pos
            img_pos = convert((w,h),(a,c,b,d))
            for i in range(len(img_pos)):
                input = str(img_pos[i])
                f_txt.write(input)
                f_txt.write(' ')
            f_txt.write('\n')


100%|██████████| 8410/8410 [36:12<00:00,  3.87it/s]
